In [2]:
#! /usr/bin/env python3

import pandas as pd
import numpy as np
import os

os.system(
    "rsync -aP raid:/home/tgrogers-raid/a/pan251/accel-sim-framework/*.csv ../../"
)


****************************************************************
*****        Multi-factor authentication required         ******
****************************************************************



receiving file list ... 
21 files to consider
platformer_2k.csv
     9656429 100%    7.02MB/s    0:00:01 (xfer#1, to-check=8/21)
platformer_4k.csv
     9634218 100%    5.46MB/s    0:00:01 (xfer#2, to-check=7/21)

sent 60 bytes  received 19295995 bytes  5513158.57 bytes/sec
total size is 74490521  speedup is 3.86


0

In [30]:
#! /usr/bin/env python3

import csv


def get_csv_data_for_merge(filepath):
    all_named_kernels = {}
    stat_map = {}
    apps = []
    cached_apps = []
    configs = []
    cached_configs = []
    stats = []
    gpgpu_build_num = None
    gpgpu_build_nums = set()
    accel_build_num = None
    accel_build_nums = set()
    data = {}
    with open(filepath, "r") as data_file:
        reader = csv.reader(data_file)  # define reader object
        state = "start"
        for row in reader:  # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                stats.append(current_stat)
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = row[1:]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        cached_apps = apps
                        cached_configs = configs
                        for config in configs:
                            count = 0
                            for appargs_kname in apps:
                                first_delimiter = appargs_kname.find("--")
                                appargs = appargs_kname[:first_delimiter]
                                kname = appargs_kname[first_delimiter + 2 :]
                                if (
                                    current_stat == "GPGPU-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    gpgpu_build_num = data[config][count][21:28]
                                    gpgpu_build_nums.add(gpgpu_build_num)
                                if (
                                    current_stat == "Accel-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    accel_build_num = data[config][count][16:23]
                                    accel_build_nums.add(accel_build_num)
                                stat_map[
                                    kname + appargs + config + current_stat
                                ] = data[config][count]
                                count += 1
                    apps = []
                    configs = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                else:
                    any_data = True
                    if accel_build_num != None and gpgpu_build_num != None:
                        full_config = (
                            row[0]
                            # + "-accel-"
                            # + str(accel_build_num)
                            # + "-gpgpu-"
                            # + str(gpgpu_build_num)
                        )
                    else:
                        full_config = row[0]
                    configs.append(full_config)
                    data[full_config] = row[1:]

    app_and_args = []
    for appargs_kname in cached_apps:
        first_delimiter = appargs_kname.find("--")
        appargs = appargs_kname[:first_delimiter]
        kname = appargs_kname[first_delimiter + 2 :]
        if appargs not in all_named_kernels:
            all_named_kernels[appargs] = []
            app_and_args.append(appargs)
        all_named_kernels[appargs].append(kname)

    # The assumption here is that every entry in each stats file is run with the same
    # git hash number, if not we are just going to warn and fail.
    if len(gpgpu_build_nums) > 1 or len(accel_build_nums) > 1:
        exit(
            "File {0} contains more than one gpgpu_build_num or accelsim_build_num - this assumes one stats file has one build num: gpgpu: {1}"
            + " accel: {2}".format(filepath, gpgpu_build_nums, accel_build_nums)
        )
    return (
        all_named_kernels,
        stat_map,
        app_and_args,
        cached_configs,
        stats,
        gpgpu_build_nums,
    )

wl = 'pbrtexture_2k'
(
    all_named_kernels,
    stat_map,
    apps_and_args,
    configs,
    stats,
    gpgpu_build_nums,
) = get_csv_data_for_merge("../../" + wl + ".csv")

# workloads = ["sponza_2k", "sponza_4k"]
# workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k"]
workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k", "sponza_2k", "sponza_4k", "materials_2k", "materials_4k", "platformer_2k", "platformer_4k"]
wl_to_name = {
    "pbrtexture_2k": "PT2",
    "pbrtexture_4k": "PT4",
    "instancing_2k": "IT2",
    "instancing_4k": "IT4",
    "render_passes_2k": "SPL2",
    "render_passes_4k": "SPL4",
    "sponza_2k": "SPH2",
    "sponza_4k": "SPH4",
    "materials_2k": "MT2",
    "materials_4k": "MT4",
    "platformer_2k": "PL2",
    "platformer_4k": "PL4"
}

stats

['Accel-Sim-build',
 'GPGPU-Sim-build',
 'gpu_tot_sim_insn\\s*=\\s*(.*)',
 'gpu_tot_sim_cycle\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_W\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[TOTAL_ACCESS\\]\\s*=\\s*(.*)',
 '\\s+Total_core_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[HIT\\]\\s*=\\s*(.*)',
 '\\s+Total_core_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[MSHR_HIT\\]\\s*=\\s*(.*)',
 '\\s+Total_core_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)',
 '\\s+Total_core_cache_sta

In [71]:
# Normalized Latency

import plotly
import plotly.graph_objects as go
import statistics

stats = {}
compute_cycles = {}
mesa_cycles = {}
compute_ipcs = {}
mesa_ipcs = {}
appconfigs = []

colors = {
    "Frame - cache": "darkslategray",
    "Frame - cache + scheduler": "deeppink",
    "Compute - cache": "darkseagreen",
    "Compute - cache + scheduler": "lime",
}
patterns = {
    "Frame - cache": "x",
    "Frame - cache + scheduler": ".",
    "Compute - cache": "-",
    "Compute - cache + scheduler": "+",
}
# ['', '/', '\\', 'x', '-', '|', '+', '.']

for wl in workloads:
    (
        all_named_kernels,
        stat_map,
        apps_and_args,
        configs,
        stats,
        gpgpu_build_nums,
    ) = get_csv_data_for_merge("../../" + wl + ".csv")

    for app in apps_and_args:
        if "all" not in app:
            continue
        for config in configs:
            appconfigs.append(app)
            compute_cycles[app + config] = 0
            mesa_cycles[app + config] = 0

    for app in apps_and_args:
        if "all" not in app:
            continue
        for config in configs:
            for kernel in all_named_kernels[app]:
                if "MESA" in kernel:
                    mesa_cycles[app + config] += int(
                        stat_map[kernel + app + config + "gpu_sim_cycle\\s*=\\s*(.*)"]
                    )
                else:
                    compute_cycles[app + config] += int(
                        stat_map[kernel + app + config + "gpu_sim_cycle\\s*=\\s*(.*)"]
                    )

gr2_to_plot = []
cp2_to_plot = []
gr1_to_plot = []
cp1_to_plot = []
x_mesa_labels = []
x_vio_labels = []
x_holo_labels = []
x_nn_labels = []
x_slam_labels = []
x_names = []
x_sizes = []

for wl in workloads:
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    x_mesa_labels.append(wl)
    x_vio_labels.append("VIO")
    x_holo_labels.append("HOLO")
    x_nn_labels.append("NN")
    x_slam_labels.append("SLAM")
    app = wl + "/all"
    config = app + "ORIN-SASS-concurrent-best-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    gr2_to_plot.append(
        1 / (mesa_cycles[config] / (mesa_cycles[baseline] + compute_cycles[baseline]))
    )
    cp2_to_plot.append(
        1
        / (compute_cycles[config] / (mesa_cycles[baseline] + compute_cycles[baseline]))
    )

    config = app + "ORIN-SASS-concurrent-VISUAL"
    gr1_to_plot.append(
        1 / (mesa_cycles[config] / (mesa_cycles[baseline] + compute_cycles[baseline]))
    )
    cp1_to_plot.append(
        1
        / (compute_cycles[config] / (mesa_cycles[baseline] + compute_cycles[baseline]))
    )

ones = [1 for i in range(len(x_mesa_labels))]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        name="Frame - cache",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=gr1_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Frame - cache']),
        marker_pattern_shape=patterns["Frame - cache"],
    )
)
fig.add_trace(
    go.Bar(
        name="Frame - cache + scheduler",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=gr2_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Frame - cache + scheduler']),
        marker_pattern_shape=patterns["Frame - cache + scheduler"],
    )
)
fig.add_trace(
    go.Bar(
        name="Compute - cache",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=cp1_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Compute - cache']),
        marker_pattern_shape=patterns["Compute - cache"],
    )
)
fig.add_trace(
    go.Bar(
        name="Compute - cache + scheduler",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=cp2_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Compute - cache + scheduler']),
        marker_pattern_shape=patterns["Compute - cache + scheduler"],
    )
)
# change width and height of the figure
fig.update_layout(
    autosize=False,
    width=1800,
    height=400,
    # title="Normalized Latency",
)
# Change the bar mode
fig.update_layout(barmode="group")
# add title in middle
# fig.update_layout(
#     title={
#         "text": "Normalized Latency",
#         "y": 0.97,
#         "x": 0.5,
#         "xanchor": "center",
#         "yanchor": "top",
#     }
# )
# change bar gap
fig.update_layout(bargap=0.1, bargroupgap=0)
# change x axis title
fig.update_layout(xaxis_title="Workloads", yaxis_title="Speedup")
# change y axis range
fig.update_yaxes(range=[0, 2], dtick=1)
# change y tick step
# change font size
fig.update_layout(xaxis=dict(tickfont=dict(size=15)))
fig.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig.update_layout(title_font_size=20)
fig.update_layout(legend=dict(font=dict(size=20)))
fig.update_layout(xaxis_title_font_size=20)
fig.update_layout(yaxis_title_font_size=20)
# change legend position to top
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.1,
    )
)
# change margin
fig.update_layout(margin=dict(l=50, r=50, t=50, b=150))

fig.show()

plotly.io.write_image(fig, "output_file.pdf", format="pdf")
plotly.io.write_image(fig, "tmp.png", format="png")


In [59]:
stats = {}
tot_ipcs = {}
appconfigs = []

for wl in workloads:
    (
    all_named_kernels,
    stat_map,
    apps_and_args,
    configs,
    stats,
    gpgpu_build_nums,
    )= get_csv_data_for_merge("../../" + wl + ".csv")

    for app in apps_and_args:
        if "all" not in app:
            continue
        for config in configs:
            appconfigs.append(app)
            tot_ipcs[app + config] = 0

    for app in apps_and_args:
        if "all" not in app:
            continue
        for config in configs:
            max_g_index = 0
            max_index = 0
            last_g_kernel = ""
            last_kernel = ""
            for kernel in all_named_kernels[app]:
                if "MESA" in kernel:
                    if int(kernel.split("-")[-1]) > max_g_index:
                        max_index = int(kernel.split('-')[-1])
                        last_g_kernel = kernel
                if int(kernel.split("-")[-1]) > max_index:
                    max_index = int(kernel.split('-')[-1])
                    last_kernel = kernel
            if config == app+"ORIN-SASS-VISUAL":
                # if baseline, select the last kernel
                tot_ipcs[app + config] = float(stat_map[last_kernel + app + config + "gpu_tot_ipc\\s*=\\s*(.*)"])
            else:
                # if not baseline, select the last MESA kernel because computes are finished already
                tot_ipcs[app + config] = float(stat_map[last_g_kernel + app + config + "gpu_tot_ipc\\s*=\\s*(.*)"])

ipc1_to_plot = []
ipc2_to_plot = []
for wl in workloads:
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    app = wl + "/all"
    config = app+"ORIN-SASS-concurrent-best-VISUAL"
    baseline = app+"ORIN-SASS-VISUAL"
    ipc1_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))

    config =app+"ORIN-SASS-concurrent-VISUAL"
    ipc2_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))


x_labels = [x_names, x_sizes]
fig = go.Figure(data=[
    go.Bar(name='Frame - cache', x=x_mesa_labels, y=ipc1_to_plot),
    go.Bar(name='Frame - cache + scheduler', x=x_mesa_labels, y=ipc2_to_plot),
])

fig = go.Figure()
fig.add_trace(
    go.Bar(
        name="Frame - cache",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=ipc1_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Frame - cache']),
        marker_pattern_shape=patterns["Frame - cache"],
    )
)
fig.add_trace(
    go.Bar(
        name="Frame - cache + scheduler",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=ipc2_to_plot + ones + ones + ones,
        # marker=dict(color=colors['Frame - cache + scheduler']),
        marker_pattern_shape=patterns["Frame - cache + scheduler"],
    )
)

# change width and height of the figure
fig.update_layout(
    autosize=False,
    width=1200,
    height=400,
    # title="Normalized Latency",
)
# Change the bar mode
fig.update_layout(barmode="group")
# add title in middle
fig.update_layout(
    title={
        "text": "Normalized Latency",
        "y": 0.97,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
# change bar gap
fig.update_layout(bargap=0.4, bargroupgap=0)
# change x axis title
fig.update_layout(xaxis_title="Workloads", yaxis_title="Speedup")
# change y axis range
# fig.update_yaxes(range=[0, 1.5], dtick=1)
# change y tick step
# change font size
fig.update_layout(xaxis=dict(tickfont=dict(size=10)))
fig.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig.update_layout(title_font_size=20)
fig.update_layout(legend=dict(font=dict(size=15)))
fig.update_layout(xaxis_title_font_size=20)
fig.update_layout(yaxis_title_font_size=20)
# change legend position to top
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.1,
    )
)
# change margin
fig.update_layout(margin=dict(l=0, r=10, t=100, b=100))

fig.show()

plotly.io.write_image(fig, "output_file.pdf", format="pdf")
plotly.io.write_image(fig, "tmp.png", format="png")